# **micrograd** autograd engine

In [1]:
import math
import random
import numpy as np

In [2]:
class Value:

    def __init__(self, data, _children=tuple(), _op=str(), label=str()):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f'Value(data={self.data})'
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, _children=(self, other), _op='+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out
    
    def __radd__(self, other):
        return self + other
    
    def __neg__(self):
        return -1 * self
    
    def __sub__(self, other):
        return self + (-other)
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, _children=(self, other), _op='*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
    def __rmul__(self, other):
        return self * other
    
    def __pow__(self, other):
        assert isinstance(other, (int, float))
        p = self.data ** other
        out = Value(p, _children=(self, ), _op=f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other-1)) * out.grad
        out._backward = _backward

        return out
    
    def __truediv__(self, other):
        return self * other**-1
    
    def exp(self):
        x = self.data
        e = math.exp(x)
        out = Value(e, _children=(self, ), _op='exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, _children=(self, ), _op='tanh')

        def _backward():
            self.grad += (1-t**2) * out.grad
        out._backward = _backward

        return out
    
    def backward(self):
        self.grad = 1.0
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                # depth-first search
                for child in v._prev:
                    build_topo(child)
                # stack
                topo.append(v)
        build_topo(self)
        topo

        for node in reversed(topo):
            node._backward()

### building out a neural net library (multi-layer perceptron) in micrograd

In [3]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

In [4]:
x = [2.0, 3.0]
n = Neuron(2)
n(x)

Value(data=0.9998885857795683)

In [5]:
class Layer:

    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

In [6]:
x = [2.0, 3.0]
n = Layer(2, 3)
n(x)

[Value(data=0.7019258590728885),
 Value(data=0.9919970748390214),
 Value(data=0.9940031985054124)]

In [7]:
class MLP:

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [8]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

Value(data=0.06624135803467011)

### creating a tiny dataset, writing the loss function

In [9]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]

ypreds = [n(x) for x in xs]
ypreds

[Value(data=0.06624135803467011),
 Value(data=0.42499476795050295),
 Value(data=0.21769898097564108),
 Value(data=0.2207241282132252)]

In [10]:
loss = sum((yout-ygt)**2 for yout, ygt in zip(ypreds, ys))
loss

Value(data=4.992576982749398)

In [11]:
loss.backward()

### collecting all of the parameters of the neural net

In [12]:
n.parameters()

[Value(data=-0.9915907498591914),
 Value(data=-0.24634985470762172),
 Value(data=-0.5707748685728351),
 Value(data=-0.3480739742625134),
 Value(data=0.9990047358234719),
 Value(data=0.5185243450292942),
 Value(data=-0.15802774376029416),
 Value(data=0.12787037488588338),
 Value(data=-0.13466682214497117),
 Value(data=0.507926467111983),
 Value(data=0.7551283399625004),
 Value(data=0.6134787282294474),
 Value(data=0.715867650451665),
 Value(data=-0.791035788125892),
 Value(data=-0.09240825253083051),
 Value(data=-0.6138945000792155),
 Value(data=0.8798290944506804),
 Value(data=-0.9375904819641496),
 Value(data=-0.07004568645975873),
 Value(data=-0.5736945156061122),
 Value(data=0.39677393574112574),
 Value(data=0.8261147306929411),
 Value(data=0.27437366982873423),
 Value(data=-0.6514206758630718),
 Value(data=0.9369610180725931),
 Value(data=-0.9290266823093556),
 Value(data=0.3059798542731631),
 Value(data=0.6899605652947725),
 Value(data=0.19242374751177005),
 Value(data=-0.60531651